In [1]:
import pandas as pd
import numpy as np
from google.colab import files

In [2]:
Upload= files.upload()
df_train= pd.read_csv('train.csv')

Saving train.csv to train.csv


In [3]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
df_train.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [6]:
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())
df_train['Embarked']= df_train['Embarked'].fillna(df_train['Embarked'].mode()[0])

#Age Categorization
conditions = [
    (df_train['Age'] < 18),
    (df_train['Age'] >= 18) & (df_train['Age'] < 60),
    (df_train['Age'] >= 60)
]

choices = ['Child', 'Adult', 'Senior']

df_train['AgeGroup'] = np.select(conditions, choices, default='Unknown')

In [7]:
df_train['Cabin'] = df_train['Cabin'].fillna('Unknown')
df_train['Deck'] = df_train['Cabin'].apply(lambda x: x[0] if x != 'Unknown' else 'U')

df_train = pd.get_dummies(df_train, columns=['Deck','Pclass','Sex','Embarked','AgeGroup'], drop_first=True)

In [8]:
df_train.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Deck_B,...,Deck_G,Deck_T,Deck_U,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S,AgeGroup_Child,AgeGroup_Senior
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,Unknown,False,...,False,False,True,False,True,True,False,True,False,False
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,False,...,False,False,False,False,False,False,False,False,False,False
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,Unknown,False,...,False,False,True,False,True,False,False,True,False,False
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,False,...,False,False,False,False,False,False,False,True,False,False
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,Unknown,False,...,False,False,True,False,True,True,False,True,False,False


In [9]:
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1
df_train['IsAlone'] = (df_train['FamilySize'] == 1).astype(int)
df_train['FarePerPerson'] = df_train['Fare'] / df_train['FamilySize']

In [10]:
df_train = pd.get_dummies(df_train, columns=['IsAlone'], drop_first=True)

In [11]:
df_train.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Deck_B,...,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S,AgeGroup_Child,AgeGroup_Senior,FamilySize,FarePerPerson,IsAlone_1
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,Unknown,False,...,False,True,True,False,True,False,False,2,3.62500,False
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,False,...,False,False,False,False,False,False,False,2,35.64165,False
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,Unknown,False,...,False,True,False,False,True,False,False,1,7.92500,True
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,False,...,False,False,False,False,True,False,False,2,26.55000,False
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,Unknown,False,...,False,True,True,False,True,False,False,1,8.05000,True


In [12]:
df_train.to_csv('clean_titanic_data.csv', index=False)

files.download('clean_titanic_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



---



---



---



In [14]:
Upload= files.upload()
df_test= pd.read_csv('test.csv')

Saving test.csv to test.csv


In [15]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [16]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [17]:
df_test.isnull().sum()

,0
PassengerId,0
Pclass,0
Name,0
Sex,0
Age,86
SibSp,0
Parch,0
Ticket,0
Fare,1
Cabin,327


In [18]:
df_test['Age'] = df_test['Age'].fillna(df_test['Age'].median())
df_test['Fare']= df_test['Fare'].fillna(df_test['Fare'].median())

#Age Categorization
conditions = [
    (df_test['Age'] < 18),
    (df_test['Age'] >= 18) & (df_test['Age'] < 60),
    (df_test['Age'] >= 60)
]

choices = ['Child', 'Adult', 'Senior']

df_test['AgeGroup'] = np.select(conditions, choices, default='Unknown')

In [19]:
df_test['Cabin'] = df_test['Cabin'].fillna('Unknown')
df_test['Deck'] = df_test['Cabin'].apply(lambda x: x[0] if x != 'Unknown' else 'U')

df_test = pd.get_dummies(df_test, columns=['Deck','Pclass','Sex','Embarked','AgeGroup'], drop_first=True)

In [20]:
df_test.head()

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Deck_B,Deck_C,...,Deck_F,Deck_G,Deck_U,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S,AgeGroup_Child,AgeGroup_Senior
0,892,"Kelly, Mr. James",34.5,0,0,330911,7.8292,Unknown,False,False,...,False,False,True,False,True,True,True,False,False,False
1,893,"Wilkes, Mrs. James (Ellen Needs)",47.0,1,0,363272,7.0000,Unknown,False,False,...,False,False,True,False,True,False,False,True,False,False
2,894,"Myles, Mr. Thomas Francis",62.0,0,0,240276,9.6875,Unknown,False,False,...,False,False,True,True,False,True,True,False,False,True
3,895,"Wirz, Mr. Albert",27.0,0,0,315154,8.6625,Unknown,False,False,...,False,False,True,False,True,True,False,True,False,False
4,896,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",22.0,1,1,3101298,12.2875,Unknown,False,False,...,False,False,True,False,True,False,False,True,False,False


In [23]:
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1
df_test['IsAlone'] = (df_test['FamilySize'] == 1).astype(int)
df_test['FarePerPerson'] = df_test['Fare'] / df_test['FamilySize']

In [25]:
df_test = pd.get_dummies(df_test, columns=['IsAlone'], drop_first=True)

In [26]:
df_test.head()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PassengerId      418 non-null    int64  
 1   Name             418 non-null    object 
 2   Age              418 non-null    float64
 3   SibSp            418 non-null    int64  
 4   Parch            418 non-null    int64  
 5   Ticket           418 non-null    object 
 6   Fare             418 non-null    float64
 7   Cabin            418 non-null    object 
 8   Deck_B           418 non-null    bool   
 9   Deck_C           418 non-null    bool   
 10  Deck_D           418 non-null    bool   
 11  Deck_E           418 non-null    bool   
 12  Deck_F           418 non-null    bool   
 13  Deck_G           418 non-null    bool   
 14  Deck_U           418 non-null    bool   
 15  Pclass_2         418 non-null    bool   
 16  Pclass_3         418 non-null    bool   
 17  Sex_male        

In [27]:
df_test.to_csv('clean_titanic_data.csv', index=False)

files.download('clean_titanic_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>